In [1]:
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
import tensorflow as tf

/Users/linbinyang/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/linbinyang/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/linbinyang/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/

In [2]:
mnist = read_data_sets("./dataset", one_hot=True, reshape=False, validation_size=0)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./dataset/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./dataset/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./dataset/t10k-images-idx3-ubyte.gz
Extracting ./dataset/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
CKPT_DIR = 'new_ckpt_dir_2'

In [5]:
class Network2(object):
    
    def __init__(self):
        self.global_step = tf.Variable(0,trainable = False)
        self.x = tf.placeholder(tf.float32, [None, 28, 28, 1], name = "x_input")
        self.label = tf.placeholder(tf.float32, [None, 10], name = "y_input")
        self.W1 = tf.Variable(tf.truncated_normal([784, 200], stddev=0.1))
        self.B1 = tf.Variable(tf.zeros([200]))
        self.W2 = tf.Variable(tf.truncated_normal([200, 100], stddev=0.1))
        self.B2 = tf.Variable(tf.zeros([100]))
        self.W3 = tf.Variable(tf.truncated_normal([100, 60], stddev=0.1))
        self.B3 = tf.Variable(tf.zeros([60]))
        self.W4 = tf.Variable(tf.truncated_normal([60, 30], stddev=0.1))
        self.B4 = tf.Variable(tf.zeros([30]))
        self.W5 = tf.Variable(tf.truncated_normal([30, 10], stddev=0.1))
        self.B5 = tf.Variable(tf.zeros([10]))
        self.XX = tf.reshape(self.x, shape=[-1, 784])
        self.Y1 = tf.nn.sigmoid(tf.matmul(self.XX, self.W1) + self.B1)
        self.Y2 = tf.nn.sigmoid(tf.matmul(self.Y1, self.W2) + self.B2)
        self.Y3 = tf.nn.sigmoid(tf.matmul(self.Y2, self.W3) + self.B3)
        self.Y4 = tf.nn.sigmoid(tf.matmul(self.Y3, self.W4) + self.B4)
        self.Ylogits = tf.matmul(self.Y4, self.W5) + self.B5
        self.Y = tf.nn.softmax(self.Ylogits, name = "z_output")
        self.cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = self.Ylogits, labels = self.label)
        self.loss = tf.reduce_mean(self.cross_entropy)*100
        self.predict = tf.equal(tf.argmax(self.Y, 1), tf.argmax(self.label, 1))
        self.accuracy = tf.reduce_mean(tf.cast(self.predict, tf.float32))
        self.train = tf.train.AdamOptimizer(0.003).minimize(self.loss, global_step = self.global_step)

In [6]:
class Train(object):
    
    def __init__(self):
        self.net = Network2()
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        
    def train(self):
        batch_size = 64
        train_step = 10000
        step = 0
        # Save Model every 1000 steps
        save_interval = 10000
        saver = tf.train.Saver(max_to_keep = 0)
        ckpt = tf.train.get_checkpoint_state(CKPT_DIR)
        # Get the current checkpoint state
        if ckpt and ckpt.get_checkpoint_state(CKPT_DIR):
            saver.restore(self.sess,ckpt.model_checkpoint_path)
            step = self.sess.run(self.net.global_step)
            print('continue from')
            print('  -> Minibatch update : ',step)
        while step < train_step:
            x,label = mnist.train.next_batch(batch_size)
            _,loss = self.sess.run([self.net.train,self.net.loss], feed_dict = {self.net.x: x,self.net.label:label})
            step = self.sess.run(self.net.global_step)
            if step % 1000 == 0:
                print('No %6d step, current loss: %.3f'%(step,loss))
            if step % save_interval == 0:
                saver.save(self.sess,CKPT_DIR + '/model',global_step = step)
    
    def calculate_accuracy(self):
        test_x = mnist.test.images
        test_label = mnist.test.labels
        acc = self.sess.run(self.net.accuracy,feed_dict = {self.net.x:test_x,self.net.label:test_label})
        print("acc: %.3f, Number of pic test: %d " % (acc, len(test_label)))

In [7]:
if __name__ == '__main__':
    model = Train()
    model.train()
    model.calculate_accuracy()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

No   1000 step, current loss: 22.411
No   2000 step, current loss: 11.512
No   3000 step, current loss: 7.652
No   4000 step, current loss: 16.620
No   5000 step, current loss: 0.992
No   6000 step, current loss: 1.078
No   7000 step, current loss: 0.408
No   8000 step, current loss: 0.187
No   9000 step, current loss: 1.735
No  10000 step, current loss: 0.893
INFO:tensorflow:new_ckpt_dir_2/model-10000 is not in all_model_checkpoint_paths. Manually adding it.
acc: 0.971, Number of pic test: 10000 


In [8]:
import numpy as np
x, label = mnist.train.next_batch(32)

In [14]:
y, y_label = mnist.train.next_batch(32)

In [9]:
print (x.shape)

(32, 28, 28, 1)


In [15]:
def model_restore_inference():
    sess = tf.Session()
    saver = tf.train.import_meta_graph('./new_ckpt_dir_2/model-10000.meta')
    saver.restore(sess, tf.train.latest_checkpoint('./new_ckpt_dir_2'))
    input_x = sess.graph.get_tensor_by_name('x_input:0')
    logits = sess.graph.get_tensor_by_name('z_output:0')
    ret = sess.run(logits, {input_x:y})
    print (ret)
    print(np.argmax(ret, axis=1))

In [16]:
model_restore_inference()

INFO:tensorflow:Restoring parameters from ./new_ckpt_dir_2/model-10000
[[9.99512076e-01 1.43652007e-10 5.55553997e-05 3.34410197e-06
  3.36648664e-05 1.97146481e-04 7.46441947e-05 1.39869928e-06
  1.32878649e-05 1.08808716e-04]
 [1.16671082e-02 4.47157217e-04 2.28046142e-02 3.04504007e-01
  1.30674758e-04 4.29497510e-02 6.48575733e-05 1.34115387e-03
  6.04988456e-01 1.11022070e-02]
 [1.47721346e-06 1.41877157e-04 1.25038612e-04 2.14566709e-04
  5.28450546e-05 1.30520064e-07 4.88322138e-09 9.98675287e-01
  5.38245695e-05 7.34864676e-04]
 [5.74654905e-06 2.90410426e-05 2.29853839e-07 2.47567550e-05
  3.36631318e-04 2.65765902e-05 6.61579591e-08 3.38238751e-04
  7.66587036e-04 9.98472154e-01]
 [7.10222002e-06 1.07815104e-05 2.24398207e-07 1.01285514e-05
  3.25616420e-04 1.72652017e-05 3.87417245e-08 2.57099076e-04
  2.10205282e-04 9.99161482e-01]
 [4.19006210e-06 1.43181547e-04 5.57044541e-06 3.50100105e-04
  5.07728646e-06 5.67332063e-05 9.19283480e-07 4.59498460e-06
  9.99266207e-01 1.6

In [17]:
unit = {}
unit['first'] = y.tolist()
import json
with open ("data3.json", "w") as fp:
    fp.write(json.dumps(unit, indent=4))